In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters

%matplotlib inline

register_matplotlib_converters()
sns.set(style="darkgrid", font_scale=1.5)

LENGTH = 600
LOOK_AHEAD_LENGTH = 40

In [2]:
print(tf.__version__)
tf.config.experimental.list_physical_devices()

2.3.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
# model = models.load_model('../../models/12kepochs500back40forward_encodedecode')


# Train Model

In [4]:
def shapeData(data, length, look_ahead_length):
    hist = []
    target = []

    for i in range(len(data)-length-look_ahead_length):
        x = data[i:i+length]
        y = data[i+length:i+length+look_ahead_length]
        hist.append(x)
        target.append(y)
    
    # Convert into numpy arrays and shape correctly (len(dataset), length) and (len(dataset), 1) respectivly
    hist = np.array(hist)
    target = np.array(target)
    hist = hist.reshape((len(hist), length, 1))

    return(hist, target)

In [5]:
def trainModel(datasets, length, model=None, quiet=False):
    for dataset in datasets:
        X_train, y_train = shapeData(dataset, length, LOOK_AHEAD_LENGTH)

        if not model:
            # Create model and compile if not already passed a model
            model = tf.keras.Sequential()
            model.add(layers.LSTM(units=20, input_shape=(length,1), dropout=0.05, activation='tanh'))
            model.add(layers.RepeatVector(LOOK_AHEAD_LENGTH))
            model.add(layers.LSTM(units=60, return_sequences=True, dropout=0.05, activation='tanh'))
            model.add(layers.TimeDistributed(layers.Dense(20)))
            model.add(layers.TimeDistributed(layers.Dense(1)))
            opt = optimizers.Adam(learning_rate=0.0001)
            model.compile(optimizer=opt, loss='mean_squared_error')
            print(model.summary())
        else:
            print("Continuing training from where we left off...")

        # Perform training
        output = 1
        if quiet:
            output = 0
        history = model.fit(X_train, y_train, epochs=1000, batch_size=64, verbose=output, shuffle=False)

        # Show loss
        if not quiet:
            loss = history.history['loss']
            epoch_count = range(1, len(loss) + 1)
            plt.figure(figsize=(6,4))
            plt.plot(epoch_count, loss, 'r--')
            plt.legend(['Training Loss'])
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.show()

    return model

In [6]:
def scaleData(paths):
    scaler = MinMaxScaler()
    datasets = []
    for path in paths:
        # perform partial fits on all datasets
        
        new_df = pd.DataFrame()
        new_df["price"] = pd.read_csv(path)[["high_price","low_price"]].mean(axis=1)
        multiple = len(new_df) - (len(new_df) % 64)
        new_df = new_df.tail(multiple)
        
        datasets.append(new_df)

        scaler = scaler.partial_fit(datasets[-1])
    for i in range(len(datasets)):
        # once all partial fits have been performed, transform every file
        datasets[i] = scaler.transform(datasets[i])
    return (datasets, scaler)

In [ ]:
model = None
for i in np.arange(1, 20):
    paths = ["../../data/ETH.csv"]
    datasets, scaler = scaleData(paths)
    model = trainModel(datasets, LENGTH, model)
    model.save(f"E-D-{i}k-600back-40forward-sm")
    print(f"Completed {i}k epochs and saved model")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20)                1760      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 40, 20)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 60)            19440     
_________________________________________________________________
time_distributed (TimeDistri (None, 40, 20)            1220      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 1)             21        
Total params: 22,441
Trainable params: 22,441
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
961/961 [==============================] - 27s 28ms/step - loss: 0.0038
Epoch 2/1000
961/961 [======

961/961 [==============================] - 27s 28ms/step - loss: 7.1858e-04
Epoch 82/1000
961/961 [==============================] - 27s 28ms/step - loss: 7.0456e-04
Epoch 83/1000
961/961 [==============================] - 27s 28ms/step - loss: 7.0157e-04
Epoch 84/1000
961/961 [==============================] - 27s 28ms/step - loss: 7.0387e-04
Epoch 85/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.9937e-04
Epoch 86/1000
961/961 [==============================] - 27s 28ms/step - loss: 7.0346e-04
Epoch 87/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.9419e-04
Epoch 88/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.9733e-04
Epoch 89/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.8833e-04
Epoch 90/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.8116e-04
Epoch 91/1000
961/961 [==============================] - 27s 28ms/step - loss: 6.8853e-04
Epoch 92/1000
961/961 [=

961/961 [==============================] - 27s 28ms/step - loss: 3.8940e-04
Epoch 172/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.9645e-04
Epoch 173/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.8300e-04
Epoch 174/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.7991e-04
Epoch 175/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.7594e-04
Epoch 176/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.7520e-04
Epoch 177/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.7669e-04
Epoch 178/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.7469e-04
Epoch 179/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.6274e-04
Epoch 180/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.6442e-04
Epoch 181/1000
961/961 [==============================] - 27s 28ms/step - loss: 3.6533e-04
Epoch 182/1000

## Test Model

In [ ]:
def testModel(model, path_to_testing_dataset, quiet=False):

    datasets, scaler = scaleData([path_to_testing_dataset])

    hist, actual = shapeData(datasets[0], LENGTH, LOOK_AHEAD_LENGTH)

    pred = model.predict(hist)

    
    # for p in pred:
    # pred_transformed.append(scaler.inverse_transform(p))
    pred_transformed = scaler.inverse_transform(pred[0])
    actual_transformed = scaler.inverse_transform(actual[0])
    hist_transformed = scaler.inverse_transform(hist[0])

    
    # print(pred[500])
    # print(hist[0])
    if not quiet:
        plt.figure(figsize=(12,8))
        plt.plot(hist_transformed, color='blue', label='History')
        plt.plot(np.arange(len(hist_transformed)-1,len(hist_transformed)+LOOK_AHEAD_LENGTH-1),pred_transformed, color='red', label='Prediction')
        plt.plot(np.arange(len(hist_transformed)-1,len(hist_transformed)+LOOK_AHEAD_LENGTH-1),actual_transformed, color='purple', label='Actual')
        plt.title('ETH Price Prediction')
        plt.legend()
        plt.show()

        pred_transformed = scaler.inverse_transform(pred[900])
        actual_transformed = scaler.inverse_transform(actual[900])
        hist_transformed = scaler.inverse_transform(hist[900])
        plt.figure(figsize=(12,8))
        plt.plot(hist_transformed, color='blue', label='History')
        plt.plot(np.arange(len(hist_transformed)-1,len(hist_transformed)+LOOK_AHEAD_LENGTH-1),pred_transformed, color='red', label='Prediction')
        plt.plot(np.arange(len(hist_transformed)-1,len(hist_transformed)+LOOK_AHEAD_LENGTH-1),actual_transformed, color='purple', label='Actual')
        plt.title('ETH Price Prediction')
        plt.legend()
        plt.show()

    return


In [ ]:
testModel(model, "../../data/ETH.csv")

# Single Prediction

In [ ]:
# For example, if we just want to predict the next timestep in the dataset we can prepare it as such:

# 1. get the [length] last points from the data set since that's what we care about
length = LENGTH
whole_df = pd.read_csv('../../data/ETH.csv')
# most_recent_period = np.array(recent_history)
recent_history = pd.DataFrame({'price': whole_df[["high_price","low_price"]].mean(axis=1)}).tail(length)
recent_history = recent_history.set_index(pd.DatetimeIndex(pd.to_datetime(whole_df['begins_at'].tail(length))))

# 2. convert to numpy array 
most_recent_period = np.array(recent_history)

# 3. normalize data
scaler = MinMaxScaler()
most_recent_period_scaled = scaler.fit_transform(most_recent_period)

# 4. reshape to the 3D tensor we expected (1, length, 1)
most_recent_period_scaled_shaped = most_recent_period_scaled.reshape((1, length, 1))

# 5. Predict
prediction = model.predict(most_recent_period_scaled_shaped)
# print(prediction[0])
# 6. Un-normalize the data
result = scaler.inverse_transform(prediction[0])
recent_history_unscaled = scaler.inverse_transform(most_recent_period_scaled)
recent_history.price = recent_history_unscaled

# print(result)
pred_df = pd.DataFrame(result)
# pred_df = pred_df.set_index(pd.DatetimeIndex(pd.date_range(recent_history.index[-1], periods=len(result[0]), freq="15s")))
plt.figure(figsize=(20,8))
plt.plot(recent_history_unscaled)
plt.plot(np.arange(length, length+LOOK_AHEAD_LENGTH),result)
plt.show()